### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
!import folium # folium need to install
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'folium'

In [2]:
pip install folium #install folium

     |████████████████████████████████| 94 kB 4.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium

### 2.Read latlong data

In [6]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d408d510692147aa9e36bcdc98528998 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ctMVclO46Ucdxch1sxNTX3KFePyEiZKtu_elWfZRfDv9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d408d510692147aa9e36bcdc98528998.get_object(Bucket='capstonefinalproject-donotdelete-pr-z1aswnj8l2q4eg',Key='mumbailatlong.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Unnamed: 0,City,Post Office,Pin Code,Latitude,Longitude
0,0,Mumbai,August Kranti Marg,400036,18.963549,72.809989
1,1,Mumbai,Aarey Milk Colony,400065,19.156129,72.870722
2,2,Mumbai,Andheri (East),400069,19.115883,72.854202
3,3,Mumbai,Andheri (West),400058,19.117249,72.833968
4,4,Mumbai,Antop Hill,400037,19.020761,72.865256


### 3.Define Foursquare Credentials

In [34]:
# define Foursquare Credentials and Version
CLIENT_ID = '15WCHXBFMWOQPNDBWH1QD1EE0USC0SJ3AJK3HPQEZCVSRM1U' # your Foursquare ID
CLIENT_SECRET = '3D3IYEUDNWGS5XILLOJAW2YFQH2LTI5INI2TDOTZGQLE0SRV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
#clean_df_new = clean_df.copy()

Your credentails:
CLIENT_ID: 15WCHXBFMWOQPNDBWH1QD1EE0USC0SJ3AJK3HPQEZCVSRM1U
CLIENT_SECRET:3D3IYEUDNWGS5XILLOJAW2YFQH2LTI5INI2TDOTZGQLE0SRV


### 4.Read explore data

In [9]:

body = client_d408d510692147aa9e36bcdc98528998.get_object(Bucket='capstonefinalproject-donotdelete-pr-z1aswnj8l2q4eg',Key='mumbaiexplore.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,Unnamed: 0,Post Office,Pin Code,Latitude,Longitude,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,0,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Doolally Taproom,18.963809,72.807695,Brewery
1,1,August Kranti Marg,400036,18.963549,72.809989,Mumbai,symphony,18.963347,72.810251,Restaurant
2,2,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Crossword,18.963474,72.807773,Bookstore
3,3,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Swati Snacks,18.966442,72.813531,Indian Restaurant
4,4,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Gustoso,18.964198,72.807726,Pizza Place


### 4.Analyze category using hotencoding

In [11]:
venues_df = df_data_2
# one hot encoding
mumbai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
mumbai_onehot['Post Office'] = venues_df['Post Office'] 
mumbai_onehot['Pin Code'] = venues_df['Pin Code'] 
mumbai_onehot['City'] = venues_df['City'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(mumbai_onehot.columns[-3:]) + list(mumbai_onehot.columns[:-3])
mumbai_onehot = mumbai_onehot[fixed_columns]

print(mumbai_onehot.shape)
mumbai_onehot.head()

(2340, 194)


,Post Office,Pin Code,City,Accessories Store,Airport Terminal,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,...,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,August Kranti Marg,400036,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,August Kranti Marg,400036,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,August Kranti Marg,400036,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,August Kranti Marg,400036,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,August Kranti Marg,400036,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
mumbai_grouped = mumbai_onehot.groupby(["Post Office", "Pin Code", "City"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped.head()

(132, 194)


,Post Office,Pin Code,City,Accessories Store,Airport Terminal,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,...,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Aarey Milk Colony,400065,Mumbai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Agashi,401301,Thane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Airoli Mode,400708,Navi Mumbai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Andheri (East),400069,Mumbai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0
4,Andheri (West),400058,Mumbai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0


### 6.Display top 5 existing facilities

In [14]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ["Post Office", "Pin Code", "City"]
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Post Office'] = mumbai_grouped['Post Office']
neighborhoods_venues_sorted['Pin Code'] = mumbai_grouped['Pin Code']
neighborhoods_venues_sorted['City'] = mumbai_grouped['City']

for ind in np.arange(mumbai_grouped.shape[0]):
    row_categories = mumbai_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]


# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(132, 8)


,Post Office,Pin Code,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aarey Milk Colony,400065,Mumbai,Fast Food Restaurant,Lake,Yoga Studio,Flea Market,Farmers Market
1,Agashi,401301,Thane,Cheese Shop,Scenic Lookout,Restaurant,Yoga Studio,Dhaba
2,Airoli Mode,400708,Navi Mumbai,Cocktail Bar,Restaurant,Garden,Yoga Studio,Dhaba
3,Andheri (East),400069,Mumbai,Hotel,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Shopping Mall
4,Andheri (West),400058,Mumbai,Indian Restaurant,Pub,Coffee Shop,Café,Gym / Fitness Center


In [15]:
# Set manually to get proper fit in the map
address = 'Mumbai'
latitude = 19.0760
longitude = 72.8777
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Mumbai are 19.076, 72.8777.


In [17]:
mumbai_merged = df_data_1
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted[["Pin Code", "1st Most Common Venue"]].set_index("Pin Code"), on="Pin Code")
print(mumbai_merged.shape)
mumbai_merged.head()

(136, 7)


,Unnamed: 0,City,Post Office,Pin Code,Latitude,Longitude,1st Most Common Venue
0,0,Mumbai,August Kranti Marg,400036,18.963549,72.809989,Coffee Shop
1,1,Mumbai,Aarey Milk Colony,400065,19.156129,72.870722,Fast Food Restaurant
2,2,Mumbai,Andheri (East),400069,19.115883,72.854202,Hotel
3,3,Mumbai,Andheri (West),400058,19.117249,72.833968,Indian Restaurant
4,4,Mumbai,Antop Hill,400037,19.020761,72.865256,Fast Food Restaurant


### 7. Visualize the data

In [84]:
my_map = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label1,common in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Post Office'],mumbai_merged['1st Most Common Venue'] ):
    labelnew =  'Post office : {} , Top Existing Infrastructure  : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='magenta',
        fill=True,
        fill_color='magenta',
        fill_opacity=0.7,
        parse_html=False).add_to(my_map)  
my_map

### 8. Business problem

In [19]:
venues_df['VenueCategory'].unique()

array(['Brewery', 'Restaurant', 'Bookstore', 'Indian Restaurant',
       'Pizza Place', 'Dessert Shop', 'History Museum',
       'Salon / Barbershop', 'Hotel', 'Bakery', 'Café', 'Coffee Shop',
       'Bar', 'Concert Hall', 'Italian Restaurant', 'Park', 'Lounge',
       'Deli / Bodega', 'Sandwich Place', 'Clothing Store',
       'Chinese Restaurant', 'Farmers Market', 'Diner', 'Food Court',
       'Cosmetics Shop', 'Food Truck', 'Fast Food Restaurant', 'Lake',
       'Shopping Mall', 'Camera Store', 'Smoke Shop', 'Bus Station',
       'Vegetarian / Vegan Restaurant', 'Pub', 'Gym / Fitness Center',
       'Bagel Shop', 'Nightclub', 'Snack Place', 'Ice Cream Shop',
       'Pharmacy', 'Grocery Store', 'Trail', 'Multiplex',
       'Parsi Restaurant', 'Irani Cafe', 'Plaza', 'Seafood Restaurant',
       'Hostel', 'Train Station', 'Outdoors & Recreation', 'Sports Club',
       'General Entertainment', 'Gym', 'College Auditorium',
       'French Restaurant', 'Gourmet Shop', 'Salad Place',
     

In [21]:
# Quality Infrastructure 
search_query= ['Restaurant', 'Hotel', 'Farmers Market', 'Shopping Mall', 'Gym / Fitness Center', 'Pharmacy',
                         'Electronics Store', 'Indie Movie Theater', 'Light Rail Station','Metro Station', 'Train','Train Station', 'Garden',
                          'Theater','ATM', 'Office', 'Bus Station', 'Bank', 'Market' , 'Business Service', 'Monument / Landmark' ,
                          'Resort', 'Hospital', 'Police Station', 'School', 'College', 'Café' , 'Park', 'Playground',
                'Convention Center', 'College Auditorium', 'Government Building', 'Airport Terminal',
                         ]
print(len(search_query))

33


In [22]:
quality_dataframe = []
quality_dataframe= venues_df.loc[venues_df['VenueCategory'].isin(search_query)]
quality_dataframe.shape

(473, 10)

In [23]:
quality_dataframe

,Unnamed: 0,Post Office,Pin Code,Latitude,Longitude,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
1,1,August Kranti Marg,400036,18.963549,72.809989,Mumbai,symphony,18.963347,72.810251,Restaurant
8,8,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Krishna Palace Residency Hotel,18.962266,72.813960,Hotel
11,11,August Kranti Marg,400036,18.963549,72.809989,Mumbai,Moshe's,18.963438,72.807810,Café
16,16,August Kranti Marg,400036,18.963549,72.809989,Mumbai,August Kranti Maidan,18.963433,72.810083,Park
18,18,August Kranti Marg,400036,18.963549,72.809989,Mumbai,di bella,18.965556,72.807004,Café
...,...,...,...,...,...,...,...,...,...,...
2327,2327,B A R C,400085,19.016700,72.850000,Mumbai,Ramee Guestline Hotel,19.017085,72.844703,Hotel
2330,2330,B A R C,400085,19.016700,72.850000,Mumbai,wadala bus depot,19.014879,72.852001,Bus Station
2332,2332,B A R C,400085,19.016700,72.850000,Mumbai,Agyari Gardens,19.018896,72.852526,Park
2334,2334,Talasari,401606,19.916700,73.233300,Thane,Jawhar Bus Station,19.919062,73.230400,Bus Station


In [24]:
# one hot encoding
qualitymumbai_onehot = pd.get_dummies(quality_dataframe[['VenueCategory']], prefix="", prefix_sep="")
# add postal, borough and neighborhood column back to dataframe
qualitymumbai_onehot['Post Office'] = quality_dataframe['Post Office'] 
qualitymumbai_onehot['Pin Code'] = quality_dataframe['Pin Code'] 
qualitymumbai_onehot['City'] = quality_dataframe['City'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(qualitymumbai_onehot.columns[-3:]) + list(qualitymumbai_onehot.columns[:-3])
qualitymumbai_onehot = qualitymumbai_onehot[fixed_columns]

print(qualitymumbai_onehot.shape)
qualitymumbai_onehot.head()
print(qualitymumbai_onehot.columns.values)

(473, 27)
['Post Office' 'Pin Code' 'City' 'Airport Terminal' 'Bank' 'Bus Station'
 'Business Service' 'Café' 'College Auditorium' 'Electronics Store'
 'Farmers Market' 'Garden' 'Government Building' 'Gym / Fitness Center'
 'Hotel' 'Indie Movie Theater' 'Light Rail Station' 'Market'
 'Monument / Landmark' 'Park' 'Pharmacy' 'Playground' 'Resort'
 'Restaurant' 'Shopping Mall' 'Theater' 'Train Station']


In [25]:
qualitymumbai_grouped = qualitymumbai_onehot.groupby(["Post Office", "Pin Code", "City"]).sum().reset_index()
print(qualitymumbai_grouped.shape)
qualitymumbai_grouped.head()

(111, 27)


,Post Office,Pin Code,City,Airport Terminal,Bank,Bus Station,Business Service,Café,College Auditorium,Electronics Store,...,Market,Monument / Landmark,Park,Pharmacy,Playground,Resort,Restaurant,Shopping Mall,Theater,Train Station
0,Agashi,401301,Thane,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Airoli Mode,400708,Navi Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Andheri (East),400069,Mumbai,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Andheri (West),400058,Mumbai,0,0,0,0,2,0,0,...,0,0,0,1,0,0,1,0,0,0
4,Anu Shakti Nagar,400094,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [26]:
qualitymumbai_grouped['Total infrastructure'] =  qualitymumbai_grouped[qualitymumbai_grouped.drop(['Post Office','Pin Code','City'], axis=1).columns.values].sum(axis=1)

In [27]:
qualitymumbai_grouped.shape

(111, 28)

### What is the best location as per infrastructure criteria?

In [28]:
qualitymumbai_grouped[qualitymumbai_grouped['Total infrastructure'] == qualitymumbai_grouped['Total infrastructure'].max()].transpose()

,9
Post Office,Bandra (West)
Pin Code,400050
City,Mumbai
Airport Terminal,0
Bank,0
Bus Station,0
Business Service,0
Café,10
College Auditorium,1
Electronics Store,1


#### Which area lack infrastructure facilities?

In [29]:
badquality = qualitymumbai_grouped[qualitymumbai_grouped['Total infrastructure'] == qualitymumbai_grouped['Total infrastructure'].min()]
badquality

,Post Office,Pin Code,City,Airport Terminal,Bank,Bus Station,Business Service,Café,College Auditorium,Electronics Store,...,Monument / Landmark,Park,Pharmacy,Playground,Resort,Restaurant,Shopping Mall,Theater,Train Station,Total infrastructure
0,Agashi,401301,Thane,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,Anu Shakti Nagar,400094,Mumbai,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
12,Bassien,401201,Thane,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
16,Bhandup (East),400042,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
19,Bhayander (East),401105,Thane,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
20,Boisar,401501,Thane,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
35,Ghansoli,400701,Navi Mumbai,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
43,Jacob Circle,400011,Mumbai,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
44,Jakegram,400606,Thane,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
45,Jawhar,401603,Thane,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 9.Potential areas for future development 

#### 9a. Choose infrastructure for which postal has best potential

In [30]:
yourchoiceinfra = 'Restaurant' # Select your choice of infrastructue from VenueCategory
badqualitychoice = qualitymumbai_grouped[qualitymumbai_grouped[yourchoiceinfra] == qualitymumbai_grouped[yourchoiceinfra].min()]
badqualitychoice['Post Office']

2                  Andheri (East)
4                Anu Shakti Nagar
6                         B A R C
7                  Ballard Estate
11                    Barve Nagar
                  ...            
102                         Vashi
103    Veer Jijamata Bhosle Udyan
106                      Vikhroli
109                        Wadala
110                         Worli
Name: Post Office, Length: 65, dtype: object

#### 9b. Choose another infrastructure for which postal has best potential

In [31]:

yourchoicearea = 'Mantralaya'   # Change with the name of postal area where you want to see potential
infraqualitychoice = qualitymumbai_grouped[qualitymumbai_grouped['Post Office'] == yourchoicearea].transpose()
infraqualitychoice = infraqualitychoice.reset_index()

In [32]:
# making data frame from csv file 
print("These are infrastructures with highest potential in" , yourchoicearea, "area : " )
for i in range(len(infraqualitychoice)) : 
    if (infraqualitychoice.iloc[i, 1] == 0):
        print(infraqualitychoice.iloc[i, 0])

These are infrastructures with highest potential in Mantralaya area : 
Airport Terminal
Bank
Bus Station
Business Service
College Auditorium
Farmers Market
Garden
Government Building
Indie Movie Theater
Light Rail Station
Market
Monument / Landmark
Park
Pharmacy
Playground
Resort
Train Station


#### Recheck your choice infrastructure at your choice postal office

In [45]:
import requests
from pandas.io.json import json_normalize
search_query = 'School'
LIMIT = 5
radius = 500
latitude = 19.016700   # Latitude of your choice postal office from 
longitude = 72.850000  # Longitude of your choice postal office from 
VERSION = 20180604
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6003790fe1508a23af783e51'},
 'response': {'venues': [{'id': '4f426fd2e4b0085fef97baaa',
    'name': 'Auxilium Convent School',
    'location': {'address': 'Wadala (W)',
     'crossStreet': 'Off Katrak Rd',
     'lat': 19.01620168738538,
     'lng': 72.85447591009165,
     'labeledLatLngs': [{'label': 'display',
       'lat': 19.01620168738538,
       'lng': 72.85447591009165}],
     'distance': 474,
     'postalCode': '400031',
     'cc': 'IN',
     'city': 'Mumbai',
     'state': 'Mahārāshtra',
     'country': 'India',
     'formattedAddress': ['Wadala (W) (Off Katrak Rd)',
      'Mumbai 400031',
      'Mahārāshtra',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d1ab941735',
      'name': 'Student Center',
      'pluralName': 'Student Centers',
      'shortName': 'Student Center',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/studentcenter_',
       'suffix': '.png'},
      'primary': True}],
    'referralId

In [46]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [49]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]
clean_dataframe.head()
clean_dataframe.tail()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Auxilium Convent School,Student Center,Wadala (W),Off Katrak Rd,19.016202,72.854476,"[{'label': 'display', 'lat': 19.01620168738538...",474,400031,IN,Mumbai,Mahārāshtra,India,"[Wadala (W) (Off Katrak Rd), Mumbai 400031, Ma...",4f426fd2e4b0085fef97baaa
1,King George School,High School,Hindu Colony,NaN,19.021661,72.848383,"[{'label': 'display', 'lat': 19.02166087460775...",577,NaN,IN,Mumbai,Mahārāshtra,India,"[Hindu Colony, Mumbai, Mahārāshtra, India]",4dbb90906e810768bf48c704
2,School,Law School,NaN,NaN,19.013098,72.849128,"[{'label': 'display', 'lat': 19.01309812095294...",411,NaN,IN,Mumbai,Mahārāshtra,India,"[Mumbai, Mahārāshtra, India]",4f19aa3be4b0a9e6db8d7446
3,Jehangir Burjorji Vaccha School For Parsi Girls,High School,NaN,NaN,19.019400,72.852462,"[{'label': 'display', 'lat': 19.01940013608133...",396,NaN,IN,NaN,NaN,India,[India],4f493bade4b0d125f728f9d5
4,J B Vacha school,None,NaN,NaN,19.018832,72.853754,"[{'label': 'display', 'lat': 19.01883249571842...",460,NaN,IN,NaN,NaN,India,[India],4c88545410523704cd00c0f1


### 10.Best place within city for major infrastructure  infrastructure

In [57]:

body = client_d408d510692147aa9e36bcdc98528998.get_object(Bucket='capstonefinalproject-donotdelete-pr-z1aswnj8l2q4eg',Key='essentialinfra.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,Unnamed: 0,Post Office,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,0,August Kranti Marg,18.963549,72.809989,Cumballa Hill Heart hospital,18.963778,72.809091,Emergency Room
1,1,Andheri (East),19.115883,72.854202,criticare hospital,19.118263,72.850639,Hospital
2,2,Andheri (West),19.117249,72.833968,Sujay Hospital,19.115915,72.834270,Hospital
3,3,Antop Hill,19.020761,72.865256,sai hospital,19.023059,72.862824,Hospital
4,4,Ballard Estate,18.936651,72.839132,Seaman Hospital,18.936524,72.839380,Medical Center


In [59]:
#remove Unnames:0
df_data_3.drop('Unnamed: 0', axis=1, inplace=True) 
df_data_3.head()

,Post Office,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,August Kranti Marg,18.963549,72.809989,Cumballa Hill Heart hospital,18.963778,72.809091,Emergency Room
1,Andheri (East),19.115883,72.854202,criticare hospital,19.118263,72.850639,Hospital
2,Andheri (West),19.117249,72.833968,Sujay Hospital,19.115915,72.834270,Hospital
3,Antop Hill,19.020761,72.865256,sai hospital,19.023059,72.862824,Hospital
4,Ballard Estate,18.936651,72.839132,Seaman Hospital,18.936524,72.839380,Medical Center


### Clustering the dataset

In [60]:
df_data_3['VenueCategory'].unique()

array(['Emergency Room', 'Hospital', 'Medical Center', 'Eye Doctor',
       'Veterinarian', "Doctor's Office", 'Bus Line', 'Sandwich Place',
       'Fast Food Restaurant', 'Fried Chicken Joint', 'Bagel Shop',
       'Snack Place', 'Seafood Restaurant', 'Indian Restaurant',
       'Food Court', 'Chinese Restaurant', 'Food Truck', 'Cafeteria',
       'Pizza Place', 'Café', 'Middle Eastern Restaurant',
       'Vegetarian / Vegan Restaurant', 'Asian Restaurant', 'Restaurant',
       'Deli / Bodega', 'Burger Joint', 'Breakfast Spot',
       'Comfort Food Restaurant', 'Dessert Shop', 'Hotel',
       'Bed & Breakfast', 'Motel', 'Boarding House', 'Resort', 'Hostel',
       'Shopping Mall', 'Building', 'Pharmacy', 'Light Rail Station',
       'Metro Station', 'Train Station', 'Platform', 'Train', 'ATM',
       'Bank', 'Campaign Office', 'Office', 'Coworking Space',
       'Tech Startup', 'Conference Room', 'Bus Station', 'Market',
       'Flea Market', 'Police Station', 'High School',
       'E

In [61]:
# one hot encoding
quality_infra_mumbai2 = df_data_3
quality_mumbai_onehot = pd.get_dummies(quality_infra_mumbai2[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
quality_mumbai_onehot['Post Office'] = quality_infra_mumbai2['Post Office'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(quality_mumbai_onehot.columns[-1:]) + list(quality_mumbai_onehot.columns[:-1])
quality_mumbai_onehot = quality_mumbai_onehot[fixed_columns]

print(quality_mumbai_onehot.shape)
quality_mumbai_onehot.head()

(1019, 81)


,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,Boarding House,Breakfast Spot,Building,...,School,Seafood Restaurant,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian
0,August Kranti Marg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Andheri (East),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Andheri (West),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Antop Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ballard Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
qualitymumbai_grouped = quality_mumbai_onehot.groupby(["Post Office"]).sum().reset_index()

print(qualitymumbai_grouped.shape)
qualitymumbai_grouped.head()

(122, 81)


,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,Boarding House,Breakfast Spot,Building,...,School,Seafood Restaurant,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Airoli Mode,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Andheri (East),1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andheri (West),1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Antop Hill,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
qualitymumbai_grouped['Total infrastructure'] =  qualitymumbai_grouped[qualitymumbai_grouped.drop(['Post Office'], axis=1).columns.values].sum(axis=1)

In [64]:
qualitymumbai_groupedmax = qualitymumbai_grouped[qualitymumbai_grouped['Total infrastructure'] == qualitymumbai_grouped['Total infrastructure'].max()]
print("Best place to stay within a city for vital infrastructure facilities :")
qualitymumbai_groupedmax[['Post Office', 'Total infrastructure']]
print(qualitymumbai_groupedmax.shape)

Best place to stay within a city for vital infrastructure facilities :
(19, 82)


In [70]:
mumbai_merged2 = qualitymumbai_grouped.copy()
mumbai_merged2 = mumbai_merged2.join(df_data_1[["Pin Code",'Latitude', 'Longitude', "Post Office" ]].set_index("Post Office"), on="Post Office")

In [71]:

fixed_columns = list(mumbai_merged2.columns[-3:]) + list(mumbai_merged2.columns[:-3])
mumbai_merged2 = mumbai_merged2[fixed_columns]

print(mumbai_merged2.shape)
mumbai_merged2.head()

(122, 85)


,Pin Code,Latitude,Longitude,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,...,Seafood Restaurant,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Total infrastructure
0,400065,19.156129,72.870722,Aarey Milk Colony,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,400708,19.172979,73.003532,Airoli Mode,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,400069,19.115883,72.854202,Andheri (East),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,8
3,400058,19.117249,72.833968,Andheri (West),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,9
4,400037,19.020761,72.865256,Antop Hill,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,6


In [75]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

mumbai_2_grouped_clustering = mumbai_merged2[["Total infrastructure"]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_2_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 0, 0, 2, 2, 0, 0, 2], dtype=int32)

In [76]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mumbai_mergedfinal = mumbai_merged2.copy()
# add clustering labels
mumbai_mergedfinal["Cluster Labels"] = kmeans.labels_
print(mumbai_mergedfinal.shape)
mumbai_mergedfinal.head() # check the last columns!

(122, 86)


,Pin Code,Latitude,Longitude,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,...,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Total infrastructure,Cluster Labels
0,400065,19.156129,72.870722,Aarey Milk Colony,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1,400708,19.172979,73.003532,Airoli Mode,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
2,400069,19.115883,72.854202,Andheri (East),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,8,0
3,400058,19.117249,72.833968,Andheri (West),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,9,0
4,400037,19.020761,72.865256,Antop Hill,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,6,0


### 11.Visualization 

In [77]:
# Set manually to get proper fit in the map
address = 'Mumbai'
latitude = 19.0760
longitude = 72.8777
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Mumbai are 19.076, 72.8777.


In [89]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'yellow',    'violet',    'orange',    'indigo', 'green']
# add markers to map
markers_colors = []
for lat, lng, label1,common, cluster in zip(mumbai_mergedfinal['Latitude'], mumbai_mergedfinal['Longitude'], mumbai_mergedfinal['Post Office'],mumbai_mergedfinal['Total infrastructure'] , mumbai_mergedfinal['Cluster Labels']):
    labelnew =  'Post office : {} , Total infrastructure : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

### 12. Check Cluster

#### 12a. Cluster 1

In [79]:
mumbai_mergedfinal.loc[mumbai_mergedfinal['Cluster Labels'] == 0]

,Pin Code,Latitude,Longitude,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,...,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Total infrastructure,Cluster Labels
2,400069,19.115883,72.854202,Andheri (East),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,8,0
3,400058,19.117249,72.833968,Andheri (West),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,9,0
4,400037,19.020761,72.865256,Antop Hill,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,6,0
7,400036,18.963549,72.809989,August Kranti Marg,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,10,0
8,400085,19.016700,72.850000,B A R C,1,0,0,0,1,0,...,0,1,1,0,0,0,0,0,10,0
10,400038,18.936651,72.839132,Ballard Estate,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,9,0
11,400051,19.061657,72.849811,Bandra (East),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,10,0
13,400090,19.168814,72.833678,Bangur Nagar,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,7,0
15,400611,19.018987,73.039095,Belapur,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,9,0
16,400078,19.143868,72.938433,Bhandup,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,10,0


#### 12b. Cluster 2

In [80]:
mumbai_mergedfinal.loc[mumbai_mergedfinal['Cluster Labels'] == 1]

,Pin Code,Latitude,Longitude,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,...,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Total infrastructure,Cluster Labels
12,400050,19.058336,72.830267,Bandra (West),1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,11,1
18,400028,18.938771,72.835335,Bhavani Shankar Road,1,0,0,0,1,1,...,0,1,0,0,0,1,0,0,13,1
24,400093,19.115287,72.861808,Chakala MIDC,1,1,0,0,1,0,...,1,0,0,0,0,0,0,1,11,1
25,400071,19.061213,72.897591,Chembur,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,11,1
28,400039,18.938771,72.835335,Council Hall,1,0,0,0,1,1,...,0,1,0,0,0,1,0,0,13,1
29,400026,18.969307,72.806538,Cumballa Hill,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,13,1
30,400014,19.019282,72.842876,Dadar,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,13,1
34,400074,18.938771,72.835335,F C I Mumbai,1,0,0,0,1,1,...,0,1,0,0,0,1,0,0,13,1
35,401206,18.938771,72.835335,Ganeshpuri,1,0,0,0,1,1,...,0,1,0,0,0,1,0,0,13,1
37,400086,19.089719,72.904597,Ghatkopar (West),1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,12,1


#### 12c. Cluster 3

In [81]:
mumbai_mergedfinal.loc[mumbai_mergedfinal['Cluster Labels'] == 2]

,Pin Code,Latitude,Longitude,Post Office,ATM,Adult Education Center,Asian Restaurant,Bagel Shop,Bank,Bed & Breakfast,...,Shopping Mall,Snack Place,Student Center,Tech Startup,Train,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Total infrastructure,Cluster Labels
0,400065,19.156129,72.870722,Aarey Milk Colony,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1,400708,19.172979,73.003532,Airoli Mode,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
5,400094,19.037528,72.928146,Anu Shakti Nagar,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
6,401302,19.202322,73.002537,Arnala,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,4,2
9,400608,19.202322,73.002537,Balcum,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,4,2
14,400084,19.095283,72.900178,Barve Nagar,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2
17,400042,19.148557,72.947066,Bhandup (East),1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,4,2
19,401101,19.197152,72.811366,Bhayandar,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
36,400701,19.119331,72.999510,Ghansoli,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,5,2
46,400011,18.981053,72.826786,Jacob Circle,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,4,2
